# INM430 Week 10 Practical - LDS file download
Scrape London Datastore .xls(s) files
Idea was to search for overlapping column names, to then cross reference with health profiles.  
Got as far as downloading files.

In [5]:
# helper functions

def getLDSDownloadLinksPageCount():
    # get the number of London Datastore pages we can scrape
    # using the same general format from week 09
    from bs4 import BeautifulSoup
    import urllib.request as ur
    urlToScrape = "https://data.london.gov.uk/dataset"
    r = ur.urlopen(urlToScrape).read()
    soup = BeautifulSoup(r, "lxml")
    # looking for the paging links found near footer
    linkList = soup.find_all('li', attrs={'class': 'dp-search__pagelink'})
    # all being well, the list will look like this (where each line is a list element)
    
    #<li class="dp-search__pagelink dp-search__pagelink--disabled"><span>«</span></li>
    #<li class="dp-search__pagelink dp-search__pagelink--active"><span>1</span></li>
    #<li class="dp-search__pagelink"><a href="/dataset?page=2">2</a></li>
    #<li class="dp-search__pagelink"><a href="/dataset?page=3">3</a></li>
    #<li class="dp-search__pagelink"><a href="/dataset?page=4">4</a></li>
    #<li class="dp-search__pagelink"><span>...</span></li>
    #<li class="dp-search__pagelink"><a href="/dataset?page=78">78</a></li>
    #<li class="dp-search__pagelink"><a href="/dataset?page=2">»</a></li>
    
    # The line we are interested in is the next to last (page 78), number 78 being
    # the text property of the link (a href attribute), which is the 6th element of
    # the linkList list starting from index 0
    try:
        iPagenums = linkList[6].text
    except:
        # string data type for consistency
        iPagenums = "0"

    return int(iPagenums)

def getLDSDownloadLinks(iPagenum):
    # each page number will have a number of links,
    # with a label (name) and a url (href)
    from bs4 import BeautifulSoup
    import urllib.request as ur
    urlToScrape = "https://data.london.gov.uk/dataset?page=" + str(iPagenum)
    r = ur.urlopen(urlToScrape).read()
    soup = BeautifulSoup(r, "lxml")
    # look for h3 headers
    linkList = soup.find_all('h3', attrs={'class': 'dp-searchresult__heading'})
    # our return list
    results = []
    for linkListItem in linkList:
        try:
            linkHeader = linkListItem.find('a', attrs={'class': "dp-searchresult__heading-link"})
            name = linkHeader.text
            href = linkHeader['href']
            ldslinks = {
                "name" : name,
                "href" : href,
            }
        except Exception as e:
            print("Error - no links found")
        results.append(ldslinks)
    return results

def getLDSFileDownloadLinks(href):
    # get the file download links - pdf, xls, etc, decide later what to do
    from bs4 import BeautifulSoup
    import urllib.request as ur
    urlToScrape = "https://data.london.gov.uk" + href
    r = ur.urlopen(urlToScrape).read()
    soup = BeautifulSoup(r, "lxml")
    download_links = soup.find_all('div', attrs={'class': 'dp-resource__indented'})
    results = []
    for download_link in download_links:
        try:
            link = download_link.find('a', attrs={'class': 'dp-resource__format'})
            fileurl = link['href']
            links = {
                "fileurl" : fileurl,
            }
            results.append(links)
        except:
            # TODO add href to error message
            print("Error occured parsing file download links for href =", href)
    # return a list of dictionaries
    return results

def checkPath(path):
    # check if path or file exist
    from pathlib import Path
    retval = False
    # remove leading forward slash
    if(path[0] == '/'):
        path = path[1:]
    p = Path(path)
    if(p.exists() == True):
        retval = True
    return retval

def makeDir(path):
    # create directory if required
    from pathlib import Path
    # remove leading forward slash
    if(path[0] == '/'):
        path = path[1:]
    p = Path(path)
    if(p.exists() == False):
        Path(path).mkdir(parents=True, exist_ok=True)
        
def jsonifier(links, toplevel):
    import json
    jsonified = "{\"" + toplevel + "\":" + json.dumps(links) + "}"
    # to keep return type consistent with xmlifier
    return str.encode(jsonified)

def checkXls(downloadlink):
    isXls = False
    filetype = downloadlink[-4:]
    filetypes = ['.xls', 'xlsx']
    if(filetype in filetypes):
        isXls = True
    return isXls

def downloadFile(linkspath, downloadpath):
    # expected values
    # linkspath ~ /dataset/migration-indicators
    # downloadpath ~ /download/migration-indicators/0db19902-5013-42af-972d-0e5481d7ac44/Long%20term%20international%20migration.xlsx
    import urllib.request
    makeDir(linkspath)
    baseurl = 'https://data.london.gov.uk/'
    # file donwload link fdlink will look like
    # https://data.london.gov.uk/download/migration-indicators/0db19902-5013-42af-972d-0e5481d7ac44/Long%20term%20international%20migration.xlsx
    fdlink = baseurl + downloadpath[1:]  
    localfile = linkspath[1:] + '/' + downloadpath.split('/')[4]
    print("Downloading", fdlink)
    print("Saving to directory", localfile)
    #urllib.request.urlretrieve(fdlink, localfile)
    
def getDownloadLinks():
    # get the number of pages
    iPagenums = getLDSDownloadLinksPageCount()
    # initialise our links dictionary
    links = []
    # get the links to pages containling download file links
    maxi = 0
    for i in range(1, iPagenums + 1):
        maxi = i
        links.extend(getLDSDownloadLinks(i))
    print("Scraped", maxi, "page links")
    # 3. Create a new entry in our links dictionary, 
    #    consisting of another dictionary with all the available files for download (pdf, xls, etc)
    for i in range (0, len(links)):
        links[i]['fileurls'] = getLDSFileDownloadLinks(links[i]['href'])
    print("Scraped file download links")
    return links

def saveDownloadLinksToJSON(links):
    myjsonfile = jsonifier(links, "links")
    f = open('lds-links.json', 'wb')
    f.write(myjsonfile)
    print("Wrote LDS download links json lds-links.json file to disk.")
    f.close()
    
def loadJSONLinks():
    import json
    f = open('lds-links.json', 'r')
    data = json.load(f)
    links = data['links']
    f.close()
    return links

In [4]:
import datetime
# Download London Datastore .xls(x) files
# Timestamp start
now = datetime.datetime.now()
print ("London datastore download scraper started:", now.strftime("%Y-%m-%d %H:%M:%S"))

# get file download links
links = getDownloadLinks()

# save json file for future reference
saveDownloadLinksToJSON(links)

# file download count
k = 0
# iterate through list of dictionaries
for i in range (0, len(links)):
    # and check the download links dictionary within dictionary
    for link in links[i]['fileurls']:
        if(checkXls(link['fileurl'])):
            k = k + 1
            downloadFile(links[i]['href'], link['fileurl'])

print("Downloaded", k, ".xls(x) files")

# Timestamp end download link scraping
now = datetime.datetime.now()
print ("London datastore download scraper scraper ended:", now.strftime("%Y-%m-%d %H:%M:%S"))            

London datastore download scraper started: 2018-12-04 16:18:54
Downloaded 80 page links
Error occured parsing file download links for href = /dataset/curio-canopy-cover-geodatabase
Error occured parsing file download links for href = /dataset/addressbase-plus-for-contractors
Downloaded page link files
Wrote LDS download links json lds-links.json file to disk.
/dataset/migration-indicators
/download/migration-indicators/0db19902-5013-42af-972d-0e5481d7ac44/Long%20term%20international%20migration.xlsx


PermissionError: [Errno 13] Permission denied: 'download/migration-indicators/0db19902-5013-42af-972d-0e5481d7ac44/Long%20term%20international%20migration.xlsx'

In [8]:
# Sanity check, download based on saved json data
import datetime
# Download London Datastore .xls(x) files
# Timestamp start
now = datetime.datetime.now()
print ("London datastore file download started:", now.strftime("%Y-%m-%d %H:%M:%S"))

# carry on
links = loadJSONLinks()

# iterate through list of dictionaries
k = 0
for i in range (0, len(links)):
    # and check the download links dictionary within dictionary
    for link in links[i]['fileurls']:
        if(checkXls(link['fileurl'])):
            k = k + 1
            downloadFile(links[i]['href'], link['fileurl'])
            print("Download number:", k)
print("Downloaded", k, ".xls(x) files")    

import datetime
# Download London Datastore .xls(x) files
# Timestamp start
now = datetime.datetime.now()
print ("London datastore file download finished:", now.strftime("%Y-%m-%d %H:%M:%S"))

London datastore file download started: 2018-12-05 10:45:48
Saving to directory dataset/medium-term-economic-forecast/GLA-london-economic-outlook-2018-11.xls
Download number: 1
Saving to directory dataset/medium-term-economic-forecast/GLA-london-economic-outlook-2018-05.xls
Download number: 2
Saving to directory dataset/medium-term-economic-forecast/GLA-london-economic-outlook-2017-11.xls
Download number: 3
Saving to directory dataset/medium-term-economic-forecast/GLA-london-economic-outlook-2017-6.xls
Download number: 4
Saving to directory dataset/medium-term-economic-forecast/gla-london-economic-outlook-2016-11.xls
Download number: 5
Saving to directory dataset/medium-term-economic-forecast/gla-london-economic-outlook-2016-5.xls
Download number: 6
Saving to directory dataset/medium-term-economic-forecast/gla-london-economic-outlook-2015-11.xls
Download number: 7
Saving to directory dataset/medium-term-economic-forecast/gla-london-economic-outlook-2015-05.xls
Download number: 8
Saving

Saving to directory dataset/public-transport-journeys-type-transport/tfl-journeys-type.xls
Download number: 67
Saving to directory dataset/london-food-strategy-consultation-2018/GLA%20consultation%20results%20-%20Unhealthy%20food%20advert%20ban.xls
Download number: 68
Saving to directory dataset/home-fire-safety-visits-by-london-borough/Home%20Fire%20Safety%20Visits%20by%20London%20Borough.xlsx
Download number: 69
Saving to directory dataset/london-fire-brigade-mobilisation-records/LFB%20Mobilisation%20data%20from%20January%202017.xlsx
Download number: 70
Saving to directory dataset/london-fire-brigade---whole-of-service-incident-summary-report/WholeOfService%20Report%20by%20calendar%20year.xls
Download number: 71
Saving to directory dataset/london-fire-brigade---whole-of-service-incident-summary-report/WholeOfService%20Report%20by%20finyear.xls
Download number: 72
Saving to directory dataset/london-fire-brigade-incident-records/LFB%20Incident%20data%20from%20January%202017.xlsx
Downlo

Saving to directory dataset/employment-rates-by-ethnicity/employment-rate-ethnic-borough.xls
Download number: 100
Saving to directory dataset/economic-inactivity/economic-inactivity.xls
Download number: 101
Saving to directory dataset/economic-inactivity/yearly-economic-inactivity-by-gender-reason.xls
Download number: 102
Saving to directory dataset/unemployment-rate-region/unemployment-region.xls
Download number: 103
Saving to directory dataset/unemployment-rate-region/unemployment-equalities-group.xls
Download number: 104
Saving to directory dataset/employment-rates/employment-rates-region-quarters-since1992.xls
Download number: 105
Saving to directory dataset/borough-council-election-results-2018/local-elections-summary-2018.xls
Download number: 106
Saving to directory dataset/borough-council-election-results-2018/london-council-2018-ward.xls
Download number: 107
Saving to directory dataset/borough-council-election-results-2018/borough-elections-2018-report-data.xls
Download number:

Saving to directory dataset/london-plan-amr14-tables-and-data/AMR%2014%20Chapter%20Two%20Tables%20.xlsx
Download number: 189
Saving to directory dataset/london-plan-amr14-tables-and-data/LDD%20-%20Housing%20Approvals%20for%20AMR14.xlsx
Download number: 190
Saving to directory dataset/london-plan-amr14-tables-and-data/LDD%20-%20Housing%20Starts%20for%20AMR14.xlsx
Download number: 191
Saving to directory dataset/london-plan-amr14-tables-and-data/LDD%20-%20Housing%20Completions%20for%20AMR14.xlsx
Download number: 192
Saving to directory dataset/london-plan-amr14-tables-and-data/LDD%20-%20Housing%20Pipeline%20for%20AMR14.xlsx
Download number: 193
Saving to directory dataset/household-waste-recycling-rates-borough/household-recycling-borough.xls
Download number: 194
Saving to directory dataset/lending-by-postcode-sector/cml-2018-q1-mortgage-lending-by-postcode.xlsx
Download number: 195
Saving to directory dataset/qualifications-economically-active-population-gender-borough/Qualifications-by

Saving to directory dataset/senior-lfepa-staff-information-2015/Senior%20LFB%20staff%20information%20-%20at%201%20April%202018.xlsx
Download number: 252
Saving to directory dataset/senior-lfepa-staff-information-2015/Senior%20LFEPA%20staff%20information%20-%20December%202015.xlsx
Download number: 253
Saving to directory dataset/senior-lfepa-staff-information-2015/lfepa-senior-staff-information-2015.xlsx
Download number: 254
Saving to directory dataset/ons-mid-year-population-estimates-custom-age-tables/ons-mye-custom-age-tool.xls
Download number: 255
Saving to directory dataset/ons-mid-year-population-estimates-custom-age-tables/ons-mye-custom-age-tool-new-ward-boundaries.xls
Download number: 256
Saving to directory dataset/ons-mid-year-population-estimates-custom-age-tables/ons-mye-custom-age-tool-ward.xls
Download number: 257
Saving to directory dataset/ons-mid-year-population-estimates-custom-age-tables/ons-mye-custom-age-tool-uk-geogs-2015.xls
Download number: 258
Saving to directo

...

Saving to directory dataset/hospital-admissions-due-injury-children-borough/child-injury-borough.xls
Download number: 646
Saving to directory dataset/2011-census-housing/trends-in-overcrowding-data.xls
Download number: 647
Saving to directory dataset/2011-census-housing/housing-characteristics-data.xls
Download number: 648
Saving to directory dataset/2011-census-housing/tenure-by-economic-activity-data.xls
Download number: 649
Saving to directory dataset/2011-census-housing/tenure-by-ns-sec-data.xlsx
Download number: 650
Saving to directory dataset/2011-census-diversity/mixed-balanced-data.xlsx
Download number: 651
Saving to directory dataset/2011-census-diversity/diversity-indices-data.xls
Download number: 652
Saving to directory dataset/2011-census-diversity/cis-ethnicity-dashboard.xlsx
Download number: 653
Saving to directory dataset/2011-census-diversity/cis-ethnicity-age-tool.xls
Download number: 654
Saving to directory dataset/2011-census-diversity/local-areas-of-ethnic-group-con

Saving to directory dataset/london-assembly-research/TNS-Internship-Survey-Aug-2014-Data.xlsx
Download number: 731
Saving to directory dataset/london-assembly-research/Talk-London-Internship-Survey-Nov-2014-Data.xlsx
Download number: 732
Saving to directory dataset/borough-council-election-results-2014/local-elections-summary-2014.xls
Download number: 733
Saving to directory dataset/borough-council-election-results-2014/london_council_election_2014_ward.xls
Download number: 734
Saving to directory dataset/borough-council-election-results-2014/borough-elections-2014-report-data.xls
Download number: 735
Saving to directory dataset/london-business-survey-2014-business-profile/LBS%202014%20BPR1%20Country%20or%20region%20of%20ownership%20of%20London%20businesses.xls
Download number: 736
Saving to directory dataset/london-business-survey-2014-business-profile/LBS%202014%20BPR2%20UK%20vs%20foreign%20ownership%20of%20London%20businesses.xls
Download number: 737
Saving to directory dataset/lond

Saving to directory dataset/lsoa-atlas/lsoa-data-old-boundaries.xls
Download number: 788
Saving to directory dataset/global-city-data/global-city-indicators.xlsx
Download number: 789
Saving to directory dataset/2011-census-assembly/census-assembly-dashboard.xls
Download number: 790
Saving to directory dataset/census-dashboards/census-dashboard.xls
Download number: 791
Saving to directory dataset/census-dashboards/census-dashboard.xlsx
Download number: 792
Saving to directory dataset/census-dashboards/ward-census-dashboard.xlsx
Download number: 793
Saving to directory dataset/census-dashboards/ward-census-dashboard-2003.xls
Download number: 794
Saving to directory dataset/census-dashboards/ward-census-dashboard-2014-wards.xlsx
Download number: 795
Saving to directory dataset/2011-census-demography/2011-census-comparison-data.xls
Download number: 796
Saving to directory dataset/2011-census-demography/london-unrounded-data.xls
Download number: 797
Saving to directory dataset/2011-census-d

Saving to directory dataset/languages-spoken-pupils-borough-msoa/languages-pupils-msoa.xls
Download number: 867
Saving to directory dataset/languages-spoken-pupils-borough-msoa/languages-pupils-borough.xls
Download number: 868
Saving to directory dataset/public-toilets/public-toilet-borough-grid.xls
Download number: 869
Saving to directory dataset/alternative-vote-referendum-results-borough/av-ref-results-borough.xls
Download number: 870
Saving to directory dataset/street-trees-borough/street-trees-borough.xls
Download number: 871
Saving to directory dataset/london-elections-results-2000/gla-elections-2000.xls
Download number: 872
Saving to directory dataset/london-elections-results-2012-wards-boroughs-constituency/gla-elections-votes-ward-2012.xls
Download number: 873
Saving to directory dataset/london-elections-results-2012-wards-boroughs-constituency/gla-elections-2012-constituency.xls
Download number: 874
Saving to directory dataset/london-elections-results-2012-wards-boroughs-cons

Saving to directory dataset/accessibility-london-underground-stations/tfl-lu-station-accessibility-object-issues.xls
Download number: 900
Saving to directory dataset/accessibility-london-underground-stations/tfl-lu-station-accessibility-barriers.xls
Download number: 901
Saving to directory dataset/accessibility-london-underground-stations/tfl-lu-station-accessibility-escalators.xls
Download number: 902
Saving to directory dataset/accessibility-london-underground-stations/tfl-lu-station-accessibility-entrances.xls
Download number: 903
Saving to directory dataset/accessibility-london-underground-stations/tfl-lu-station-accessibility-toilets.xls
Download number: 904
Saving to directory dataset/accessibility-london-underground-stations/tfl-lu-station-accessibility-car-parks.xls
Download number: 905
Saving to directory dataset/accessibility-london-underground-stations/tfl-lu-station-accessibility-help-points.xls
Download number: 906
Saving to directory dataset/accessibility-london-undergrou

In [192]:
# as an aside, links could be rebuilt from json file
# download excel files
# Validate links rebuilding from json file
# import json
# f = open('lds-links.json', 'r')
# data = json.load(f)
# links = data['links']
# f.close()

# Extras, sanity checks

In [14]:
# this is the path on website where download link is to be found e.g.
# https://data.london.gov.uk/dataset/diversity-london-report-data
path = '/dataset/diversity-london-report-data'
# note we will use the same path locally to store the downloaded file

# this is the file donwload location e.g.
# https://data.london.gov.uk/download/diversity-london-report-data/66ee75f0-3424-4333-8ebf-d227bc74b562/diversity-in-london-data.xlsx
downloadlink = '/download/diversity-london-report-data/66ee75f0-3424-4333-8ebf-d227bc74b562/diversity-in-london-data.xlsx'
# So we check if the file exists locally e.g.
# /dataset/diversity-london-report-data/66ee75f0-3424-4333-8ebf-d227bc74b562/diversity-in-london-data.xlsx'
# downloadlink = downloadlink.split('/')
downloadFile(url, localPath)

NameError: name 'url' is not defined

In [16]:
# another sanity check
localPath = '/dataset/medium-term-economic-forecast'
url = '/download/medium-term-economic-forecast/ec246c96-7e35-4661-820f-da01ab123d08/gla-london-economic-outlook-2015-11.xls'
downloadFile2(url, localPath)

PermissionError: [Errno 13] Permission denied: 'download/medium-term-economic-forecast/ec246c96-7e35-4661-820f-da01ab123d08/gla-london-economic-outlook-2015-11.xls'

In [262]:
path[1:]

'dataset/diversity-london-report-data'

In [235]:
localDowloadPath = path + '/' + downloadlink.split('/')[4]
localDowloadPath[1:]

'dataset/diversity-london-report-data/diversity-in-london-data.xlsx'

In [234]:
downloadlink.split('/')[4]

'diversity-in-london-data.xlsx'